# EDF - Code your Future Hackathon

## Introduction

Welcome to the EDF - Code Your Future Hackathon. 

#### Problem Statement: You have a dataset, this is a raw dataset obtained from numerous properties in the UK. You are trying to create an algorithm to predict the heating load of a property. You do not need to understand the science behind heating loads, energy transfer etc. Try focus more on the data rather than the context, though, some context does help. 

In a real life situation, you will never get clean data. You will have anomalies, nulls, or data that just doesn't make sense.

This data has been purposely edited to reflect some of that.
- It has random data
- Anomalies
- Null values
- Messy data in general

Your task is to clean and explore this data using various data manipulation techniques in python that we will teach you. Once you have got the data to a good state, you can then pass it through a Machine Learning Model. This notebook is a skeleton of steps, with some hard steps taken for you, but feel free to change or manipulate any existing code to really understand it.

Steps are as followed:
- Clean and explore the data
- Pass it through the Machine learning model

At the end of this, we will give each team a score, to see who did the best prediction model. Remember, its not just the model that affects how well you predict things. Data cleaning and manipulation play a massive role. So even if you dont want to understand the maths of a model, cleaning and manipulating data, makes all the difference. 

## This is the skeleton notebook

### Importing the relevant packages
Feel free to install or include more packages in your project, just run this cell again. 

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import KFold
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.preprocessing import StandardScaler

### Functions
These are some functions that we have designed for you to use that would help as tools. In your task. 

In [5]:

# Recode surface area and extract useful text from wall_description
def extract_figures(text, position = -3):
    # text <- string to extract information from
    # position <- integer that tells the position of number in string
    listOfWords = text.split()
    word = listOfWords[position]
    return word

def create_correlation_map(df):
    plt.figure(figsize=(16, 6))
    # Store heatmap object in a variable to easily access it when you want to include more features (such as title).
    # Set the range of values to be displayed on the colormap from -1 to 1, and set the annotation to True to display the correlation values on the heatmap.
    heatmap = sns.heatmap(df.corr(), vmin=-1, vmax=1, annot=True)
    # Give a title to the heatmap. Pad defines the distance of the title from the top of the heatmap.
    heatmap.set_title('Correlation Heatmap', fontdict={'fontsize':12}, pad=12);

def pretty_print_coefficients(model, columns):
    # Function to display estimated coefficients in a more human-readable way

    # coef -> np.array: estimated regression coefficients
    # columns -> list of str: columns used for prediction

    coefDf = pd.DataFrame({"Feature": columns,"Coefficients": model.coef_[0]})
    coefDf.head(len(columns))
    return coefDf

### Loading in the data

In [6]:
# Load in the energy.csv data
df = pd.read_csv("energy_noisy_train.csv")

The data is now inside the notebook, and you can do all sorts of manipulation on the data by calling the variable df. Try running the cell below to see the data that's now saved in the notebook. Of course, you can call it other than 'df' (convention to denote dataframe).

In [7]:
df.head()

,Unnamed: 0,Relative Compactness,Roof Area,Overall Height,Orientation,Glazing Area,Glazing Area Distribution,Heating Load,Location,surface_area_categories,wall_description,energxe
0,223,0.71,220.5,3.5,5,0.10,4.407175,10.72,Mountain,High Surface Area,The wall colour is green and the area is 269.5...,571.0
1,285,0.62,220.5,3.5,3,0.10,3.725392,12.74,City,Very High Surface Area,The wall colour is indigo and the area is 367....,NaN
2,350,0.82,147.0,7.0,4,0.25,1.530090,25.38,Countryside,Low Surface Area,The wall colour is orange and the area is 318....,NaN
3,363,0.74,220.5,3.5,5,0.25,2.007144,12.30,City,Average Surface Area,The wall colour is orange and the area is 245....,395.0
4,551,0.76,122.5,7.0,5,0.40,1.649852,40.43,Seaside,Average Surface Area,The wall colour is indigo and the area is 416....,NaN


## Exploratory Data Analysis & Cleaning

This is part one of the task, we need to do an (exploratory data analysis) EDA as well as clean the dataset.

Hints/steps to take

- Identify the data attributes, explore the data. How long is it, how many rows are there. What columns, what datatypes are those columns? hints(Use: df.shape and df.dtypes)  
- A ML model cannot have categories that are too unique, if a category has (red, blue, green) thats three unique categories, but what about 50? Thats' too many unless you want the model to be complicated. Possible but not reasonable.
- What should we do with the null data?
- Do the continuous (numbered) columns have any relationship/correlation? explore this
- Explore the categorical data, plot bar charts, is there anything interesting/not important.
- Extra: What's interesting about the orientation column, is there something else we could do to it based on its context
- Extra: The sky is the limit, do as much manipulation/exploration as you want to get the data in a good state

#### Initial Explorations

Wall Description has 56 unique values in that column, this cannot be used as a categorical column. This column can be cleaned by isolating colour and number. As we spot this, we can do this now.

In [10]:
# We utilise the extract figures function we defined before and used a .apply on the columns to do this. 

df["Wall Area"] = df["wall_description"].apply(lambda x: float(extract_figures(x, -3)))
df["Colour"] = df["wall_description"].apply(lambda x: extract_figures(x, 4))
df.head()

#### Null explorations

Now we check the nulls.

Hints 
Possible to use: df.isnull().sum()

Make decisions based on this and possibly use: df.dropna() 
(Be weary when using this to make sure you don't lose too much data if you do this)

In [11]:
# Checking the Null values graphically 
sns.heatmap( ___ ,yticklabels=False, cmap='viridis')

### Some more EDA

Now data seems clean at a glance, we should start doing explorations to see if it can be cleaned more/if theres some redundant stuff in there. 

In [12]:
# This is a function that we defined earlier, be sure to look at this function to see if you understand it. 
# It creates a correlation map where 1 means perfect correlation. This is to identify relationships.
create_correlation_map(df)


The unnamed column seems useless, and it has a high relationship with glazing area, so if we have glazing area, we don't need unnamed. It seems to be a redundant column that may mess with our data. We can choose to remove this now, or test it with the model later. In this case we decide to remove it now.

Lets look at what has a high correlation with heating load. It seems height, and roof area is very important to predict heating load. So we should make sure these columns are cleaned and note their good relationship.

In [ ]:
# Remove 'Unnamed: 0' column from the data
df.drop(columns = ___ , inplace = True)

Now, check the relationships graphically.

Hint: Use sns.pairplot(df)

Take a look at the graphs above! DO you notice something on the Relative Compactness, Roof Area, Overall Height and Orientation graphs?


Those are likely extreme values and should be removed before proceeding to fit any machine learning model. While we're at it - there are 2 extreme observations for the variables we want to predict (Heating load) that we should remove as well.

Let's check some anomalies using boxplots!

#### Plotting anomalies

Hint: Use sns.boxplot(x=df['COLUMN NAME']) 

In [14]:
#EXAMPLE OF BOXPLOT

sns.boxplot(x=df['Relative Compactness'])

Do you think there are other extreme values which should be removed?

#### Let's explore some categorical features

Hints
- Use sns.countplot
- Use sns.barplot
- Use any more visualisations to understand the categories!

Don't forget to use google to see the documentation!

With categorical feature, you see that colour is just random, you could see if it can be used in the model, but in this case since logically colour doesn't seem to have an effect, and from below, they all seem equal volumes. Therefore, you could remove colour without major drawbacks.


In [16]:
plt.figure(figsize=(10,7))
ax = sns.countplot(x=___, data=df)

You could also plot average heating load by loaction to get a better understanding of the relationship between these two variables. Now let's see the bar plot with heating load (what we are predicting). Feel free to add in/experiment with other exploratory variable candidates!

In [17]:
# Plotting average heating load by location
plt.figure(figsize=(10,7))
location_heating_average_rating  = df.groupby(by =[___]).mean().reset_index()
ax=sns.barplot(x=___, y=___, data=location_heating_average_rating)

## Machine Learning models

Congratulations! Now you should have a good idea of the data and cleaned it ready to pass through a machine learning model. 

Everything in this section is prefilled apart from DUMMY VARIABLES.

We are keeping most of this section more as a playground section as an introductory.

#### Before the ML model, we must turn all the categorical columns into dummy variables.

In [19]:
# Create dummy variables out of surface area categories
colNames = ["Very Low Surface Area", "Low Surface Area", "Average Surface Area", "High Surface Area", "Very High Surface Area"]
df[colNames] = pd.get_dummies(df["surface_area_categories"],dtype=int)


# Create dummy columns for location variable
colNames = ["Mountain", "City", "___side", "Seaside", "Lakeside"]
df[colNames] = pd.get_dummies(df[___],dtype=int)

# Drop redundant columns
df.drop(labels=[___, ____], axis=1, inplace=True)
df.head()

### EXTRA: could there be any other columns in the dataset that could be treated as a categorical column? What can you do.

Orientation is a categorical column despite having integers, this is important as it would skew the model if it was treated as numerical. For example, logically, orientation 1 is the same as orientation 2, but in numerical turns, orientation 2 is more positive, when they are at the same level but mean different things. Hence, we should make this categorical and also make dummy variables

In [20]:
# Create dummy values for Orientation
colNames = ["North Orientation", "East Orientation", "South Orientation", "West Orientation"]
df[colNames] = pd.___(df[___],dtype=int)
# Drop redundant columns
df.drop(labels=[____], axis=1, inplace=True)
df.head()

### Linear Regression Model

Below we added all the variables we think could be highly relevant to the ML model - and your job is to find the best ones among them! We also added a variable for the number of folds (how many groups the data should be split into) which you can use to reduce or increase the training set.
We also added the option to change the random seed. This changes which observations go to which group - a good model predicts accurately, no matter what random seed value is given to it.

After you make changes to the model, make sure to re-run the 3 cells below to update the model results.

In [ ]:
# Explain the variables below to understand what they can vary. 

# Playground area - edits these and see how the model Linear Regression model changes!
depVars = ['Relative Compactness', 
'Roof Area', 
'Overall Height', 
#"North Orientation",
#"East Orientation",
#"South Orientation",
#"West Orientation" 
'Glazing Area', 
'Glazing Area Distribution', 
#"Mountain",
#"City",
#"Countryside",
#"Seaside",
#"Lakeside"
'Wall Area', 
'Very Low Surface Area', 
'Low Surface Area', 
'Average Surface Area', 
'High Surface Area', 
'Very High Surface Area']

outcomeVar = "Heating Load"

numberOfFolds = 3
randomSeed = 42

In [ ]:
# Now build the Linear Model from settings above
X = pd.DataFrame(df[depVars])
Y = pd.DataFrame(df[outcomeVar])

model = LinearRegression()
rScores = list()
kFold = KFold(n_splits=numberOfFolds, shuffle=True, random_state=randomSeed)

for _, (train, test) in enumerate(kFold.split(X, Y)):
    model.fit(X.iloc[train,:], Y.iloc[train,:])
    score = model.score(X.iloc[test,:], Y.iloc[test,:])
    rScores.append(score)

rScores

[0.9729227623373188, 0.9699769716590502, 0.9748411721612356]

In [ ]:
# Check estimated coefficients
pretty_print_coefficients(model, depVars)

,Feature,Coefficients
0,Relative Compactness,34.835775
1,Roof Area,0.281596
2,Overall Height,11.640947
3,Glazing Area,19.886372
4,Glazing Area Distribution,0.059157
5,Wall Area,0.054888
6,Very Low Surface Area,3.074971
7,Low Surface Area,2.645203
8,Average Surface Area,-8.686788
9,High Surface Area,3.828052


The outcome from the 2nd cell is the r-squared scores of the models from each test run. The higher those numbers are, the better. It reflects that your model explains more of the variation that is seen in the data and generally results in better predictions.

The final cell gives you the value for each coefficient from the last run. It is generally a good idea to take a look at these and see which variables are positively/negatively affecting the predicted outcome.

Below, you can play around with prediction - using the estimated coefficients from the model, you can see what heating load your model would predict for an actual house. You can even get some observations from the actual dataset and see how your predictions differ from the actual value. Remember - your model would not be judged on 1 prediction, so it is better to be correct in general than being extra precise for 1 use case.

Note: You should use the exact same variables for prediction (in the same order!) as you used when training your dataset.

### Predicting example

In [ ]:
# Put in Markdown explanation

# Use trained model for prediction
newData = pd.DataFrame({
'Relative Compactness': 0.8, 
'Roof Area': 150, 
'Overall Height': 7, 
#"North Orientation": 0,
#"East Orientation": 0,
#"South Orientation": 1,
#"West Orientation": 0,
'Glazing Area': 0.45, 
'Glazing Area Distribution': 5, 
#"Mountain": 0,
#"City": 1,
#"Countryside": 0,
#"Seaside": 0,
#"Lakeside": 0,
'Wall Area': 500, 
'Very Low Surface Area': 0, 
'Low Surface Area': 0, 
'Average Surface Area': 1, 
'High Surface Area': 0, 
'Very High Surface Area': 0
}, index=[0])

model.predict(newData)

array([[41.59782143]])

#### Ridge Regression

By the time you read this, you should be familiar with all but the last 2 options/parameters you see in the cell below (if not, please read the description above the section for linear regression). The final 2 parameters control some specifics of the ridge regression:

penaltyParameter signifies how much negative score to give for large coefficients. If this value is increased, some estimated parameters (generally those that are the worst when it comes to prediction) will tend towards zero. Thus, this is a great way to see what variables are most important for your predictions.

standardizeData is a boolean indicating if data should be standardized. Standardization is beyond the scope of this Hackathon (although Google/YouTube is your friend if you want to know more about it) so the only thing you need to know is that if you set this to "True" then all potential variables are given an equal penalty. If set to "False", then variables which are the furthest away from 0 are given greater penalty than those which are closer to zero. I recommend leaving this as "True".

In [ ]:
# Playground area - edits these and see how the Ridge Regression model changes!
depVars = ['Relative Compactness', 
'Roof Area', 
'Overall Height', 
#"North Orientation",
#"East Orientation",
#"South Orientation",
#"West Orientation" 
'Glazing Area', 
'Glazing Area Distribution', 
#"Mountain",
#"City",
#"Countryside",
#"Seaside",
#"Lakeside"
'Wall Area', 
'Very Low Surface Area', 
'Low Surface Area', 
'Average Surface Area', 
'High Surface Area', 
'Very High Surface Area']

outcomeVar = "Heating Load"
numberOfFolds = 3
randomSeed = 42
penaltyParameter = 1
standardizeData = True

In [ ]:
# Takes care of data standardization process (only run once)
scaler = StandardScaler()

X = pd.DataFrame(df[depVars])
Y = pd.DataFrame(df[outcomeVar])

if standardizeData:
    scaler.fit(X.values)
    scaledData = scaler.transform(X)
    standardDf = pd.DataFrame(scaledData, index=X.index, columns=X.columns)

else:
    standardDf = df

In [ ]:
# Fit actual Ridge Regression model
model = Ridge(alpha=penaltyParameter)
rScores = list()
kFold = KFold(n_splits=numberOfFolds, shuffle=True, random_state=randomSeed)

for _, (train, test) in enumerate(kFold.split(X, Y)):
    model.fit(X.iloc[train,:], Y.iloc[train,:])
    score = model.score(X.iloc[test,:], Y.iloc[test,:])
    rScores.append(score)

rScores

[0.9722850150166756, 0.9644943773497127, 0.9742457719154707]

In [ ]:
# Ridge regression coefficients = higher values indicate better correlation between predictor and outcome
pretty_print_coefficients(model, depVars)

,Feature,Coefficients
0,Relative Compactness,0.375911
1,Roof Area,0.173863
2,Overall Height,10.580892
3,Glazing Area,17.153368
4,Glazing Area Distribution,0.093342
5,Wall Area,0.019649
6,Very Low Surface Area,2.696185
7,Low Surface Area,1.909377
8,Average Surface Area,-7.758811
9,High Surface Area,3.164169


The outcome from the 2nd cell is the r-squared scores of the models from each test run. The higher those numbers are, the better. It reflects that your model explains more of the variation that is seen in the data and generally results in better predictions.

The final cell gives you the value for each coefficient from the last run. It is generally a good idea to take a look at these and see which variables are positively/negatively affecting the predicted outcome. If any of these are close to zero, maybe they are not that relevant and should be removed after all.

Below, you can play around with prediction - using the estimated coefficients from the model, you can see what heating load your model would predict for an actual house. You can even get some observations from the actual dataset and see how your predictions differ from the actual value. Remember - your model would not be judged on 1 prediction, so it is better to be correct in general than being extra precise for 1 use case.

Note: You should use the exact same variables for prediction (in the same order!) as you used when training your dataset

Are the Ridge Regression predictions different from the Linear Regression predictions? If so, how? Is any of the two consistently better or worse?

In [ ]:
# Prediction using Ridge Regression
# Use trained model for prediction
newData = pd.DataFrame({
'Relative Compactness': 0.8, 
'Roof Area': 150, 
'Overall Height': 7, 
#"North Orientation": 0,
#"East Orientation": 0,
#"South Orientation": 1,
#"West Orientation": 0,
'Glazing Area': 0.45, 
'Glazing Area Distribution': 5, 
#"Mountain": 0,
#"City": 1,
#"Countryside": 0,
#"Seaside": 0,
#"Lakeside": 0,
'Wall Area': 500, 
'Very Low Surface Area': 0, 
'Low Surface Area': 0, 
'Average Surface Area': 1, 
'High Surface Area': 0, 
'Very High Surface Area': 0
}, index=[0])

if standardizeData:
    newData = scaler.transform(newData)

model.predict(newData)

array([[-57.71722828]])